<a href="https://colab.research.google.com/github/alex-pakalniskis/gisc606-spring2023/blob/main/lab1/GISC606_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GISC 606 Lab 1: Normalized Burn Ratio (NBR)

## Overview
* Part 1: Assess burn scars with satellite imagery and open-source GIS tools
* Part 2: Assigned readings and literature search
* Your Deliverable

## Part 1
Assess burn scars with satellite imagery
> Note: This tutorial was originally developed by Esri's Learn Team. You can find the official maintained version at [this location](https://learn.arcgis.com/en/projects/assess-burn-scars-with-satellite-imagery/). Additionally you can find other tutorials in the [tutorial gallery](https://learn.arcgis.com/en/gallery/).


You'll use Linux command line tools and Python to display multispectral imagery data and calculate a Normalized Burn Ratio spectral index. No prior Linux and minimal prior Python experience is required (I've provided all the code).


In [ ]:
# Install dtrx, an extraction utility that simplifies user experience. 
# In a more real-life scenario you wouldn't necessarily do this step every time but only once. 
# In Google Colab, we get a fresh computer each time so need to reinstall each time. 
# https://pypi.org/project/dtrx/ 
!pip install dtrx

In [ ]:
# Download the lab data from Esri
# We're only using the provided imagery, so will throw away the ArcGIS Pro project and other related files.
# https://man7.org/linux/man-pages/man1/wget.1.html
!wget -c https://www.arcgis.com/sharing/rest/content/items/1bc2bc1305b447fa939b937a8867114f/data -O MontanaFires

In [ ]:
# Extract the compressed MontanaFires file into an uncompressed folder
!dtrx MontanaFires

In [ ]:
# Move a nested subdirectory in the extracted MontanaFires.1 directory to a new location 
# https://man7.org/linux/man-pages/man1/mv.1.html
!mv MontanaFires.1/commondata/raster_data raster_data

In [ ]:
# Clean up your workspace by removing unnecessary folders and files
# https://man7.org/linux/man-pages/man1/rm.1.html
!rm MontanaFires
!rm -rf MontanaFires.1

In [ ]:
# Display the contents of `raster_data` directory
# https://man7.org/linux/man-pages/man1/ls.1.html
!ls raster_data/

In [ ]:
# Install dependencies for rasterio then install rasterio
!sudo apt-get install python-numpy gdal-bin libgdal-dev
!pip install rasterio

In [ ]:
# Check the version of rasterio installed via command line
!rio --version

In [ ]:
# Use rasterio command line tool to display image metadata for 2014 pre-fire imagery
!rio info raster_data/G_2014.tif --indent 2 --verbose

In [ ]:
# Do the same thing for the 2015 post-fire imagery but save to a JSON file
!rio info raster_data/G_2015.tif --indent 2 --verbose > G_2015.json

In [ ]:
# Let's switch to Python now
# Notice how we stop using ! since we're no longer trying to invoke Linux command line tools
# We're importing the `json` library and opening the JSON file we just created with rio
import json

with open("G_2015.json") as f:
  info_2015 = json.load(f)

In [ ]:
# Let's display the JSON "dictionary keys"
info_2015.keys()

In [ ]:
# And we can display a specific key value
info_2015["descriptions"]

In [ ]:
# Determine the CRS of 2015 imagery data here


## Question 1 (double-click this cell and write your answer below the question)
### Part A
What are the band names for the 2015 imagery data?
* aka `info_2015["descriptions"]`

### Part B
Using the blank code cell above, determine the CRS of the 2015 imagery data. You can copy+paste the preceeding code cell, but make sure to replace "description" with "crs".

In [ ]:
# Import `rasterio` and some support libraries for performing array computation (multispectral images are big arrays) and visualizing data
import rasterio
from matplotlib import pyplot as plt
from rasterio.plot import show, show_hist
import numpy as np

In [ ]:
# Load the data set into rasterio
data_14 = rasterio.open("raster_data/G_2014.tif")
data_15 = rasterio.open("raster_data/G_2015.tif")

In [ ]:
# Plot one of the bands
plt.imshow(data_15.read(3), cmap="pink")
plt.show()

In [ ]:
# Apply super basic contrast stretching
img=data_15.read(3)
vmin, vmax = np.nanpercentile(img, (5,95))  # 5-95% stretch
img_plt = plt.imshow(img, cmap='pink', vmin=vmin, vmax=vmax)
plt.show()

In [ ]:
# Convenience function to normalize data to 0-1 scale
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

In [ ]:
# Load RGB data from 2015 into memory, normalize, then display
red = data_15.read(3)
green = data_15.read(2)
blue = data_15.read(1)

redn = normalize(red)
greenn = normalize(green)
bluen = normalize(blue)

rgb = np.dstack((redn, greenn, bluen))
plt.imshow(rgb)

In [ ]:
# Calculate NBR for 2014 and display the result
band_5_14 = data_14.read(5)
band_7_14 = data_14.read(7)

n5 = normalize(band_5_14)
n7 = normalize(band_7_14)

nbr_14 = (n5 - n7) / (n5 + n7)

plt.imshow(nbr_14, cmap="gray")
plt.show()

In [ ]:
# Calculate NBR for 2015 and display the result
band_5_15 = data_15.read(5)
band_7_15 = data_15.read(7)

n5 = normalize(band_5_15)
n7 = normalize(band_7_15)

nbr_15 = (n5 - n7) / (n5 + n7)



plt.imshow(nbr_15, cmap="gray")
plt.show()

In [ ]:
# Derive NBR difference then plot it
nbr_delta = nbr_15 - nbr_14

plt.imshow(nbr_delta, cmap="RdYlGn")
plt.show()

In [ ]:
# Convenience function to apply standard deviation contrast stretch to an array
import math
# https://stackoverflow.com/questions/59813350/create-new-raster-tif-from-standard-deviation-stretched-bands-works-with-dst
def std_stretch_data(data, n=2):
    """Applies an n-standard deviation stretch to data."""

    # Get the mean and n standard deviations.
    mean, d = data.mean(), data.std() * n

    # Calculate new min and max as integers. Make sure the min isn't
    # smaller than the real min value, and the max isn't larger than
    # the real max value.
    new_min = math.floor(max(mean - d, data.min()))
    new_max = math.ceil(min(mean + d, data.max()))

    # Convert any values smaller than new_min to new_min, and any
    # values larger than new_max to new_max.
    data = np.clip(data, new_min, new_max)

    # Scale the data.
    data = (data - data.min()) / (new_max - new_min)
    return data


In [ ]:
# Apply standard deviation contrast stretching to enhance NBR difference raster then display
std_stretched = std_stretch_data(nbr_delta)
plt.imshow(std_stretched, cmap="RdBu")
plt.show()

In [ ]:
# Display stretched data and a corresponding histogram
fig, (axrgb, axhist) = plt.subplots(1, 2, figsize=(14,7))
show(std_stretched, ax=axrgb, cmap="RdBu")
show_hist(std_stretched, bins=50, histtype='stepfilled', lw=0.0, stacked=False, alpha=0.3, ax=axhist)
plt.show()

In [ ]:
# Save NBR difference raster to file
with rasterio.open(
    "nbr_delta.tif", 
    "w", 
    driver="GTiff", 
    height=data_15.shape[0], 
    width=data_15.shape[1], 
    dtype=red.dtype, 
    count=1, 
    crs='+proj=latlong', 
    transform=data_15.transform
    ) as dst:
  dst.write(nbr_delta, 1)

In [ ]:
# Display directory contents
!ls

## Question 2:
Please use `rio` to display information about the `nbr_delta.tif` data set to your command line output. Use the empty code cell below this question.

## Part 2
### Assigned readings
* [Mapping Burns and Natural Reforestation using Thematic Mapper Data by Lopez Garcia, M., and V. Caselles](https://www.researchgate.net/publication/246761915_Mapping_burns_and_natural_reforestation_using_Thematic_Mapper_data)
* [Landscape Assessment: Remote Sensing of Severity, the Normalized Burn Ratio; and Ground Measure of Severity, the Composite Burn Index by Key, C. and N. Benson, N](https://www.researchgate.net/publication/241687027_Landscape_Assessment_Ground_measure_of_severity_the_Composite_Burn_Index_and_Remote_sensing_of_severity_the_Normalized_Burn_Ratio)

## Your Deliverable
### Part 1
Save a copy of your Notebook in the GitHub repository you created during Lab 0. 
* Click File > Download > Download .ipynb
* Navigate to your GitHub repo (created during Lab 0) and make sure that you're logged into GitHub
* Click Add file > Upload files
* Select the .ipynb file you saved
* Paste the link to your uploaded file in the GitHub issue you opened last week in this repo: https://github.com/alex-pakalniskis/gisc606-spring2023



### Part 2
Please write a 2-4 sentence summary (paraphrase in your own words) about each article then compile your results into JSON format. Follow the steps below to submit your JSON data.
> Note: The steps may contain links with additional explanations to provide you with missing context.

1. Login to your GitHub account if you aren't already logged in
1. Navigate to [https://github.com/alex-pakalniskis/gisc606-spring2023](https://github.com/alex-pakalniskis/gisc606-spring2023)
1. [Create a branch](https://docs.github.com/en/pull-requests/collaborating-with-pull-requests/proposing-changes-to-your-work-with-pull-requests/creating-and-deleting-branches-within-your-repository#creating-a-branch-using-the-branch-dropdown) of `alex-pakalniskis/gisc606-spring2023` from `main` branch using the branch dropdown
    1. Title your branch `lab 1: your csulb email without @csulb.edu`, i.e. mine would be `lab 1: alex.pakalniskis`
1. Create a file called `your-email-without-@csulb.edu.json`, i.e. mine would be `alex-pakalniskis.json` in the `gisc606-spring/lab1` subdirectory.
1. Copy the contents of Part 2's submission template (see below) into `your-email-without-@csulb.edu.json` then update with your information. 
1. [Merge your changes](https://docs.github.com/en/pull-requests/collaborating-with-pull-requests/proposing-changes-to-your-work-with-pull-requests/creating-a-pull-request#creating-the-pull-request) into `main` branch by opening a pull request
1. [Request a review](https://docs.github.com/en/pull-requests/collaborating-with-pull-requests/proposing-changes-to-your-work-with-pull-requests/requesting-a-pull-request-review) from @alex-pakalniskis

#### Part 2 Submission Template

``` json
{
    "your_name": "Alex Pakalniskis",
    "your_csulb_email": "alex.pakalniskis@csulb.edu",
    "articles": [
      {
        "title": "Mapping Burns and Natural Reforestation using Thematic Mapper Data",
        "authors": "Lopez Garcia, M., and V. Caselles",
        "link": "https://www.researchgate.net/publication/246761915_Mapping_burns_and_natural_reforestation_using_Thematic_Mapper_data",
        "your_summary": "In two to four sentences, paraphrase the content of the article (your own words)."
      },
      {
        "title": "Landscape Assessment: Remote Sensing of Severity, the Normalized Burn Ratio; and Ground Measure of Severity, the Composite Burn Index.",
        "authors": "Key, C. and N. Benson, N",
        "link": "https://www.researchgate.net/publication/241687027_Landscape_Assessment_Ground_measure_of_severity_the_Composite_Burn_Index_and_Remote_sensing_of_severity_the_Normalized_Burn_Ratio",
        "your_summary": "In two to four sentences, paraphrase the content of the article (your own words)."
      }
    ]
}
```

